In [59]:
import pandas as pd
import numpy as np
import google_analytics_reporting_api as ga
import json as json
import datetime

In [60]:
# Input
Start_Date='2017-10-22'
End_Date=str(datetime.datetime.now().date()-datetime.timedelta(days=1)) # Yesterday
writer_Category="/home/jian/Projects/Tone_Networks/Output/Tone_Network_GA_Category_Data_From_2017-10-22_To_"+End_Date+".xlsx"

In [61]:
writer_Category=pd.ExcelWriter(writer_Category, engine='xlsxwriter')
analytics = ga.initialize_analyticsreporting('/home/jian/Projects/Tone_Networks/GA/Tone Networks - prod-00ada014c4ba.json')
general_matric=['ga:sessions','ga:users','ga:bounces','ga:bounceRate','ga:pageviews','ga:uniquePageviews','ga:totalEvents','ga:uniqueEvents']

In [62]:
def num_func_trans(df):
    if 'ga:sessions' in df.columns:
        df['ga:sessions']=df['ga:sessions'].astype(int)
    if 'ga:users' in df.columns:
        df['ga:users']=df['ga:users'].astype(int)
    if 'ga:bounces' in df.columns:
        df['ga:bounces']=df['ga:bounces'].astype(int)
    if 'ga:bounceRate' in df.columns:
        df['ga:bounceRate']=round(df['ga:bounceRate'].astype(float)/100,4)
    if 'ga:pageviews' in df.columns:
        df['ga:pageviews']=df['ga:pageviews'].astype(int)
    if 'ga:uniquePageviews' in df.columns:
        df['ga:uniquePageviews']=df['ga:uniquePageviews'].astype(int)
    if 'ga:totalEvents' in df.columns:
        df['ga:totalEvents']=df['ga:totalEvents'].astype(int)
    if 'ga:uniqueEvents' in df.columns:
        df['ga:uniqueEvents']=df['ga:uniqueEvents'].astype(int)
    if 'ga:goalCompletionsAll' in df.columns:
        df['ga:goalCompletionsAll']=df['ga:goalCompletionsAll'].astype(int)
    if 'ga:dateHourMinute' in df.columns:
        df['ga:dateHourMinute']=df['ga:dateHourMinute'].apply(lambda x:x[0:4]+"-"+x[4:6]+"-"+x[6:8]+" "+x[8:10]+":"+x[10:12])
    if 'ga:date' in df.columns:
        df['ga:date']=df['ga:date'].apply(lambda x:x[0:4]+"-"+x[4:6]+"-"+x[6:8])
    
    return df

In [44]:
Start_Minute=datetime.datetime.strptime(Start_Date,'%Y-%m-%d')
End_Minute=datetime.datetime.strptime(str(datetime.datetime.now().date()),'%Y-%m-%d')
All_Minutes=pd.date_range(start=Start_Minute,end=End_Minute,freq="Min")


In [45]:
Start_Minute=datetime.datetime.strptime(Start_Date,'%Y-%m-%d')
End_Minute=datetime.datetime.strptime(str(datetime.datetime.now().date()),'%Y-%m-%d')
All_Minutes=pd.date_range(start=Start_Minute,end=End_Minute,freq="Min")

def Add_Zero_Minute(df):

    df['dateHourMinute']=df['dateHourMinute'].apply(lambda x: datetime.datetime.strptime(x,'%Y-%m-%d %H:%M'))
    Empty_Time_Df=pd.DataFrame(columns=df.columns,index=range(len(All_Minutes)))
    Empty_Time_Df['dateHourMinute']=All_Minutes
    Time_Df_Notin=Empty_Time_Df[~Empty_Time_Df['dateHourMinute'].isin(df['dateHourMinute'])]
    df=df.append(Time_Df_Notin)
    df.reset_index(inplace=True)
    del (df['index'])
    df=df.sort_values('dateHourMinute')
    df.reset_index(inplace=True)
    del (df['index'])    
    return df  

In [46]:
Start_Hour=datetime.datetime.strptime(Start_Date,'%Y-%m-%d')
End_Hour=datetime.datetime.strptime(str(datetime.datetime.now().date()),'%Y-%m-%d')
All_Hours=pd.date_range(start=Start_Minute,end=End_Minute,freq="H")

def Add_Zero_Hour(df):
    df['dateHour']=df['dateHour'].apply(lambda x: datetime.datetime.strptime(x,'%Y%m%d%H'))
    Empty_Time_Df=pd.DataFrame(columns=df.columns,index=range(len(All_Hours)))
    Empty_Time_Df['dateHour']=All_Hours
    Time_Df_Notin=Empty_Time_Df[~Empty_Time_Df['dateHour'].isin(df['dateHour'])]
    df=df.append(Time_Df_Notin)
    df.reset_index(inplace=True)
    del (df['index'])
    df=df.sort_values('dateHour')
    df.reset_index(inplace=True)
    del (df['index'])
    return df


In [47]:
Event_Mapping=pd.read_excel("/home/jian/Projects/Tone_Networks/event map.xlsx")
Event_Mapping.columns=['eventCategory','eventCategory_New']
Event_Mapping['eventCategory']=Event_Mapping['eventCategory'].apply(lambda x:x[14:len(x)])
Event_Mapping['eventCategory_New']=Event_Mapping['eventCategory_New'].apply(lambda x:x[14:len(x)])

In [48]:
# Event_Minute
Event_Time=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=['ga:totalEvents','ga:uniqueEvents'], ga_dimensions=['ga:dateHourMinute','ga:eventCategory','ga:eventAction','ga:eventLabel'])
Event_Time=num_func_trans(Event_Time)
Event_Time=Event_Time[['ga:dateHourMinute','ga:eventCategory','ga:eventAction','ga:eventLabel']+['ga:totalEvents','ga:uniqueEvents']]
Event_Time.rename(columns=lambda x:x[3:len(x)],inplace=True)
Event_Time=Add_Zero_Minute(Event_Time)
# Event_Time['eventAction']=Event_Time['eventAction'].replace(0,"No Event")
# Event_Time['eventCategory']=Event_Time['eventCategory'].replace(0,"No Event")
# Event_Time['eventLabel']=Event_Time['eventLabel'].replace(0,"No Event")
Event_Time_Mapped=pd.merge(Event_Time,Event_Mapping,on="eventCategory",how="left")
Event_Time_Mapped.to_excel(writer_Category,sheet_name="Event_all_label",index=False)

Event_Time=Event_Time[['dateHourMinute','eventCategory','totalEvents','uniqueEvents']]
Event_Time_Agg=pd.merge(Event_Time,Event_Mapping,on="eventCategory",how="left")
Event_Time_Agg=Event_Time_Agg.groupby(['dateHourMinute','eventCategory_New'])['totalEvents','uniqueEvents'].sum()
Event_Time_Agg=Event_Time_Agg.reset_index()
Event_Time_Agg=Event_Time_Agg.sort_values(['dateHourMinute','eventCategory_New'])
Event_Time_Agg.to_excel(writer_Category,sheet_name="Event_Category",index=False)


In [49]:
# Minute Page Path
Dimension=['ga:dateHourMinute','ga:pagePath']
Minute_Page_Path=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=Dimension)
Minute_Page_Path=num_func_trans(Minute_Page_Path)
Minute_Page_Path=Minute_Page_Path[Dimension+general_matric]

Minute_Page_Path.rename(columns=lambda x:x[3:len(x)],inplace=True)
Minute_Page_Path=Add_Zero_Minute(Minute_Page_Path)
Minute_Page_Path.to_excel(writer_Category,"Minute_Page_Path",index=False)


In [50]:
# Minute MediaSource
Dimension=['ga:dateHourMinute','ga:sourceMedium']
Minute_SourceMedium=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=Dimension)
Minute_SourceMedium=num_func_trans(Minute_SourceMedium)
Minute_SourceMedium=Minute_SourceMedium[Dimension+general_matric]

Minute_SourceMedium.rename(columns=lambda x:x[3:len(x)],inplace=True)
Minute_SourceMedium=Add_Zero_Minute(Minute_SourceMedium)
Minute_SourceMedium.to_excel(writer_Category,"Minute_SourceMedium",index=False)


In [51]:
# Minute Metro_City
Dimension=['ga:dateHourMinute','ga:metro','ga:city']
Minute_Metro_City=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=Dimension)
Minute_Metro_City=num_func_trans(Minute_Metro_City)
Minute_Metro_City=Minute_Metro_City[Dimension+general_matric]

Minute_Metro_City.rename(columns=lambda x:x[3:len(x)],inplace=True)
Minute_Metro_City=Add_Zero_Minute(Minute_Metro_City)
Minute_Metro_City.to_excel(writer_Category,"Minute_Metro_City",index=False)


In [52]:
# Minute Lag_Long
Dimension=['ga:dateHourMinute','ga:latitude','ga:longitude']
Minute_Lag_Long=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=Dimension)
Minute_Lag_Long=num_func_trans(Minute_Lag_Long)
Minute_Lag_Long=Minute_Lag_Long[Dimension+general_matric]

Minute_Lag_Long.rename(columns=lambda x:x[3:len(x)],inplace=True)
Minute_Lag_Long=Add_Zero_Minute(Minute_Lag_Long)
Minute_Lag_Long.to_excel(writer_Category,"Minute_Lag_Long",index=False)


In [53]:
# Hour_AgeBracket
Dimension=['ga:dateHour','ga:userAgeBracket']
Hour_AgeBracket=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=Dimension)

Hour_AgeBracket=num_func_trans(Hour_AgeBracket)
Hour_AgeBracket=Hour_AgeBracket[Dimension+general_matric]
Hour_AgeBracket.rename(columns=lambda x:x[3:len(x)],inplace=True)
Hour_AgeBracket=Add_Zero_Hour(Hour_AgeBracket)
Hour_AgeBracket.to_excel(writer_Category,"Hour_AgeBracket",index=False)


In [54]:
# Hour_Gender
Dimension=['ga:dateHour','ga:userGender']
Hour_Gender=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=Dimension)
Hour_Gender=num_func_trans(Hour_Gender)
Hour_Gender=Hour_Gender[Dimension+general_matric]
Hour_Gender.rename(columns=lambda x:x[3:len(x)],inplace=True)
Hour_Gender=Add_Zero_Hour(Hour_Gender)
Hour_Gender.to_excel(writer_Category,"Hour_Gender",index=False)


In [55]:
# Hour_interestInMarketCategory
Dimension=['ga:dateHour','ga:interestInMarketCategory']
Hour_interestInMarketCategory=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=Dimension)
Hour_interestInMarketCategory=num_func_trans(Hour_interestInMarketCategory)
Hour_interestInMarketCategory=Hour_interestInMarketCategory[Dimension+general_matric]
Hour_interestInMarketCategory.rename(columns=lambda x:x[3:len(x)],inplace=True)
Hour_interestInMarketCategory=Add_Zero_Hour(Hour_interestInMarketCategory)
Hour_interestInMarketCategory.to_excel(writer_Category,"Hour_interestInMarketCategory",index=False)


In [56]:
# Hour_interestAffinityCategory
Dimension=['ga:dateHour','ga:interestAffinityCategory']
Hour_interestAffinityCategory=ga.get_report_over_10000_rows(analytics=analytics, VIEW_ID='158084228',start_date=Start_Date, end_date=End_Date, 
                               ga_metrics=general_matric, ga_dimensions=Dimension)
Hour_interestAffinityCategory=num_func_trans(Hour_interestAffinityCategory)
Hour_interestAffinityCategory=Hour_interestAffinityCategory[Dimension+general_matric]
Hour_interestAffinityCategory.rename(columns=lambda x:x[3:len(x)],inplace=True)
Hour_interestAffinityCategory=Add_Zero_Hour(Hour_interestAffinityCategory)
Hour_interestAffinityCategory.to_excel(writer_Category,"Hour_interestAffinityCategory",index=False)


In [57]:
writer_Category.save()

""
